In [1]:
import numpy as np
import math
import matplotlib as plt
import pandas as pd
import csv
import math
from numpy import linspace, zeros

In [2]:
#First define the constants
G = 6.678e-11
c = 3.0e+8
pi = math.pi
#currently taking natural system of units
#G = c = 1
#declare all the necessary arrays(inputs and outputs)
#write the function with RK4 code
#declare all the output files
#store all the arrays into these files
#plot if required

In [3]:
#RK4 method to solve a given pair of ODEs in a fixed range of the independent parameter
def RK4_complete(N, ri , rf , dy_dr, dz_dr, yi, zi):
    
    h = (rf - ri)/N
    r = np.zeros((N))
    y = np.zeros((2,N))

    for i in range(N):
        r[i] = r[i] + i*h
        
        if(i == 0):
            y[0][i] = yi
            y[1][i] = zi
            
        else:
            y[0][i] = y[0][i-1] + (K1 + 2*K2 + 2*K3 + K4)/6
            y[1][i] = y[1][i-1] + (L1 + 2*L2 + 2*L3 + L4)/6
        
        print(dy_dr(r[i], y[0][i], y[1][i]), " ", dz_dr(r[i], y[0][i], y[1][i]))
        K1 = dy_dr(r[i], y[0][i], y[1][i])
        L1 = dz_dr(r[i], y[0][i] , y[1][i])

        K2 = dy_dr(r[i] + h/2, y[0][i] + h*K1/2, y[1][i] + h*L1/2)
        L2 = dz_dr(r[i] + h/2, y[0][i] + h*K1/2, y[1][i] + h*L1/2)

        K3 = dy_dr(r[i] + h/2, y[0][i] + h*K2/2, y[1][i] + h*L2/2)
        L3 = dz_dr(r[i] + h/2, y[0][i] + h*K2/2, y[1][i] + h*L2/2)

        K4 = dy_dr(r[i] + h, y[0][i] + h*K3, y[1][i] + h*L3)
        L4 = dz_dr(r[i] + h, y[0][i] + h*K3, y[1][i] + h*L3)
        
        print(i, " ", y[0][i], " ", y[1][i])
        print(K1, L1)
        print(K2, L2)
        print(K3, L3)
        print(K4, L4)
        print("****************")
    return(y)
print("All good until here")

All good until here


In [4]:
def RK4_step(dy_dr, dz_dr, yi, zi, r, h):
    
    K1 = dy_dr(r, yi,  zi)
    L1 = dz_dr(r, yi , zi)
    #print("K1 and L1 done", r, yi, zi)
    
    K2 = dy_dr(r + h/2, yi + h*K1/2, zi + h*L1/2)
    L2 = dz_dr(r + h/2, yi + h*K1/2, zi + h*L1/2)
    #print("K2 and L2 are done", r + h/2, yi + h*K1/2, zi + h*L1/2)
    #print("here is the der. ", dz_dr(r + h/2, yi + h*K1/2, zi + h*L1/2))
    K3 = dy_dr(r + h/2, yi + h*K2/2, zi + h*L2/2)
    L3 = dz_dr(r + h/2, yi + h*K2/2, zi + h*L2/2)
    #print("K3 and L3 are done", r + h/2, yi + h*K2/2, zi + h*L2/2)
    #print(dy_dr(r + h/2, yi + h*K2/2, zi + h*L2/2))
    K4 = dy_dr(r + h, yi + h*K3, zi + h*L3)
    L4 = dz_dr(r + h, yi + h*K3, zi + h*L3)
    #print("K4 and L4 are done", r + h/2, yi + h*K3, zi + h*L3)
    yi = yi + h*(K1 + 2*K2 + 2*K3 + K4)/6
    zi = zi + h*(L1 + 2*L2 + 2*L3 + L4)/6
    #print((K1 + 2*K2 + 2*K3 + K4)/6, (L1 + 2*L2 + 2*L3 + L4)/6)
    #print(K2, L2)
    #print(K3, L3)
    #print(K4, L4)
    #print(yi, zi)
    return(yi, zi)


In [5]:
ri = 0.0
rf = 20.00
mi = 0.0
#epsi = 1e+15 * 7.4237e-19  #natural units
epsi = 5e+34 #SI units
N = 20000
#K = 12.236 #natural units
K = 5380 #Si units

#define the EoS function to provide eps(r) and P(r) for every r
def P(eps):
    return K*(eps)**(5/3)

Pi = P(epsi)
print("Pi :", Pi)
# def eps(P_r):
#     eps = P_r**(3/5)

#define the derivative function to get dm/dr and dP/dr
def dm_dr(r, m_r, P_r):
    eps = (1/K)*(P_r)**0.6
    #print(eps)
    mprime = 4*pi*eps*(r**2)/c**2
    return mprime

def dP_dr_units(r, m_r, P_r):
    eps = (1/K)*P_r**(0.6)
    Pprime = -(G/(c**2*r))*(eps + P_r)*((m_r*c**2 + 4*pi*r**3*P_r)/(r*c**2 - 2*G*m_r + 1e-20))
    return Pprime
#print(Pprime(ri, mi, pi))

def dP_dr_nat(r, m_r, P_r):
    eps = (1/K)*P_r**(0.6)
    Pprime = -(eps + P_r)*((m_r + 4*pi*(r**3)*P_r)/(r*r - 2*m_r*r + 1e-20))
    return Pprime

r = linspace(ri,rf, N)
h = r[1] - r[0]
y = np.zeros((2,N))

for i in range(N):
    r[i] = r[i]*1000 #for SI units
    if(i == 0):
        y[0][i] = mi
        y[1][i] = Pi
        print(i, "  ",r[i], "  ", y[0][i], "  ", y[1][i])
    else: 
        if(y[1][i-1] >= 1e-20):
            #print(y[1][i-1], "proceed")
            #r[i] = ri + i*h
            y[0][i], y[1][i] = RK4_step(dm_dr,dP_dr_units, y[0][i-1], y[1][i-1] , r[i], h)  
            print(i, "  ",r[i], "  ", y[0][i], "  ", y[1][i])
        else:
            print(y[1][i-1], "Boundary reached")
            print(i, "  ", r[i], "  ",  y[0][i], "  ", y[1][i])
            break
mf = y[0][N-1]
rf = r[N-1]
Pf = y[1][N-1]
print(mf,rf)
        
#RK4_complete(N, ri, rf, mPrime, Pprime, mi, Pi)


Pi : 3.650891694320171e+61
0    0.0    0.0    3.650891694320171e+61
1    1.000050002500125    nan    nan
nan Boundary reached
2    2.00010000500025    0.0    0.0
0.0 20.0


/tmp/ipykernel_6214/1947213846.py:21: RuntimeWarning: invalid value encountered in double_scalars
  eps = (1/K)*(P_r)**0.6
/tmp/ipykernel_6214/1947213846.py:27: RuntimeWarning: invalid value encountered in double_scalars
  eps = (1/K)*P_r**(0.6)


In [6]:
np.savetxt("TOV_mass_radius_SI.txt", np.c_[r, y[0], y[1]])

In [7]:
#Written by Debabrata Da
import numpy as np
import math
import matplotlib as plt
import pandas as pd
from numpy import linspace, zeros

pi = math.pi

# def p(r,m,p):
#     press = k*rh**(5/3)
#     return press

def dm_dr(r, m, p):
    rh = (1/k)*(p)**(3/5)
    mprime = 4*pi*rh*(r**2)
    return mprime

def dp_dr(r, m, p):
    rh = (1/k)*(p)**(3/5)
    Pprime = -(rh + p)*(m + 4*pi*(r**3)*p)/(r*r - 2*m*r + 1e-20)
    return Pprime


def RK4_step(r, m, p, h):
    y = np.zeros(2)
    
    K1 = dm_dr(r, m,  p)
    L1 = dp_dr(r, m , p)    
    
    K2 = dm_dr(r + h/2, m + h*K1/2, p + h*L1/2)
    L2 = dp_dr(r + h/2, m + h*K1/2, p + h*L1/2)
    
    K3 = dm_dr(r + h/2, m + h*K2/2, p + h*L2/2)
    L3 = dp_dr(r + h/2, m + h*K2/2, p + h*L2/2)
    
    K4 = dm_dr(r + h, m + h*K3, p + h*L3)
    L4 = dp_dr(r + h, m + h*K3, p + h*L3)
    
    y[0] = m + h*(K1 + 2*K2 + 2*K3 + K4)/6
    y[1] = p + h*(L1 + 2*L2 + 2*L3 + L4)/6
    
    #print(y[0], y[1])
    return(y)

N = 20000
r = linspace(0, 20, N)
h = r[1] - r[0]

m = np.zeros(N)
p = np.zeros(N)
rh = np.zeros(N)


r[0] = 0.0
m[0] = 0.0
rh[0] = 1e15*7.4237e-19 #km^-2
k = 12.236
p[0] = k*rh[0]**(5/3)
print(p[0])



for i in range(N-1):
    [m[i+1] , p[i+1]] = RK4_step(r[i], m[i], p[i], h)
    #rh[i+1] = (p[i+1]/k)**(3/5)
    print(r[i+1], m[i+1], p[i+1])
    if p[i+1] <= 1e-10:
        mf = m[i]
        rf = r[i]
        break
    
print(mf/1.989e+30,rf)

7.447436753623798e-05
0.001000050002500125 1.1421374699905435e-12 7.447436712604253e-05
0.00200010000500025 9.137099718968521e-12 7.447436604089999e-05
0.0030001500075003754 3.0837711287455703e-11 7.447436423687443e-05
0.0040002000100005 7.309679623790042e-11 7.44743617120532e-05
0.005000250012500625 1.4276717791594602e-10 7.447435846610593e-05
0.006000300015000751 2.467016787115076e-10 7.44743544989394e-05
0.0070003500175008756 3.917531198198421e-10 7.447434981051933e-05
0.008000400020001 5.847743210026168e-10 7.447434440083075e-05
0.009000450022501126 8.326181003489784e-10 7.447433826986637e-05
0.01000050002500125 1.1421372740366213e-09 7.447433141762234e-05
0.011000550027501376 1.5201846560928571e-09 7.447432384409651e-05
0.012000600030001501 1.9736130581556827e-09 7.447431554928765e-05
0.013000650032501625 2.5092752892348494e-09 7.447430653319507e-05
0.014000700035001751 3.1340241554729317e-09 7.44742967958184e-05
0.015000750037501875 3.854712459906397e-09 7.447428633715747e-05
0.0

0.9300465023251163 0.0009173020466266673 7.416300959935714e-05
0.9310465523276165 0.0009202612824145415 7.416234086489817e-05
0.9320466023301165 0.0009232268660945901 7.416167141696578e-05
0.9330466523326166 0.00092619880441623 7.416100125557671e-05
0.9340467023351168 0.0009291771041286563 7.416033038074775e-05
0.9350467523376169 0.0009321617719808427 7.415965879249569e-05
0.9360468023401171 0.0009351528147215403 7.415898649083732e-05
0.9370468523426172 0.0009381502390992781 7.415831347578948e-05
0.9380469023451173 0.0009411540518623629 7.415763974736901e-05
0.9390469523476174 0.000944164259758878 7.415696530559278e-05
0.9400470023501175 0.0009471808695366843 7.415629015047766e-05
0.9410470523526177 0.0009502038879434193 7.415561428204054e-05
0.9420471023551178 0.0009532333217264967 7.415493770029836e-05
0.9430471523576179 0.000956269177633107 7.415426040526802e-05
0.9440472023601181 0.0009593114624102165 7.415358239696649e-05
0.9450472523626182 0.000962360182804567 7.415290367541074e-

1.3810690534526726 0.0029981633493267874 7.378929530655885e-05
1.3820691034551729 0.0030046666167177303 7.378830676522786e-05
1.383069153457673 0.003011179249938779 7.378731751977774e-05
1.3840692034601731 0.0030177012556156426 7.378632757023321e-05
1.3850692534626732 0.003024232640373703 7.378533691661902e-05
1.3860693034651732 0.0030307734108380147 7.378434555895992e-05
1.3870693534676735 0.003037323573633305 7.378335349728068e-05
1.3880694034701735 0.0030438831353839743 7.378236073160609e-05
1.3890694534726737 0.003050452102714094 7.378136726196097e-05
1.3900695034751738 0.0030570304822474083 7.378037308837014e-05
1.391069553477674 0.0030636182806073327 7.377937821085844e-05
1.392069603480174 0.0030702155044169542 7.377838262945074e-05
1.393069653482674 0.0030768221602990313 7.377738634417193e-05
1.3940697034851743 0.003083438254875993 7.37763893550469e-05
1.3950697534876744 0.0030900637947699396 7.377539166210056e-05
1.3960698034901746 0.0030966987866026417 7.377439326535784e-05
1.

1.9880994049702485 0.008911985544093264 7.306076281589134e-05
1.9890994549727488 0.008925379064591213 7.305935206532746e-05
1.9900995049752488 0.00893878590408723 7.305794062883595e-05
1.991099554977749 0.008952206068966503 7.305652850645195e-05
1.992099604980249 0.008965639565613756 7.30551156982107e-05
1.9930996549827493 0.008979086400413248 7.305370220414735e-05
1.9940997049852494 0.008992546579748772 7.305228802429715e-05
1.9950997549877494 0.009006020110003655 7.305087315869536e-05
1.9960998049902496 0.00901950699756076 7.304945760737721e-05
1.9970998549927497 0.009033007248802481 7.304804137037799e-05
1.99809990499525 0.009046520870110748 7.304662444773297e-05
1.99909995499775 0.009060047867867023 7.304520683947747e-05
2.00010000500025 0.009073588248452301 7.304378854564683e-05
2.00110005500275 0.009087142018247113 7.304236956627637e-05
2.0021001050052503 0.00910070918363152 7.304094990140145e-05
2.0031001550077505 0.009114289750985115 7.303952955105745e-05
2.0041002050102508 0.0

2.191109555477774 0.011912688416835971 7.276036635089683e-05
2.1921096054802742 0.011928915955437016 7.275881710129371e-05
2.193109655482774 0.01194515809939048 7.275726717324775e-05
2.1941097054852743 0.011961414854982606 7.275571656679753e-05
2.1951097554877745 0.01197768622849913 7.275416528198165e-05
2.196109805490275 0.011993972226225277 7.275261331883872e-05
2.1971098554927746 0.01201027285444576 7.275106067740736e-05
2.198109905495275 0.012026588119444782 7.274950735772623e-05
2.199109955497775 0.012042918027506035 7.274795335983398e-05
2.200110005500275 0.012059262584912701 7.274639868376928e-05
2.201110055502775 0.01207562179794745 7.274484332957083e-05
2.2021101055052754 0.01209199567289244 7.274328729727735e-05
2.2031101555077757 0.012108384216029318 7.274173058692756e-05
2.2041102055102755 0.012124787433639218 7.27401731985602e-05
2.2051102555127757 0.012141205332002764 7.273861513221403e-05
2.206110305515276 0.012157637917400066 7.273705638792783e-05
2.207110355517776 0.01

...

17.870893544677234 3.7199085530730587 1.3530861035630342e-05
17.871893594679733 3.7203018107323187 1.3528055237085688e-05
17.872893644682236 3.720695063467248 1.3525249818047683e-05
17.873893694684735 3.7210883112719166 1.3522444778515066e-05
17.874893744687235 3.7214815541403947 1.3519640118486565e-05
17.875893794689734 3.7218747920667528 1.3516835837960889e-05
17.876893844692237 3.7222680250450617 1.351403193693674e-05
17.877893894694736 3.722661253069393 1.3511228415412801e-05
17.878893944697236 3.7230544761338185 1.3508425273387747e-05
17.879893994699735 3.72344769423241 1.3505622510860236e-05
17.880894044702234 3.72384090735924 1.3502820127828915e-05
17.881894094704737 3.724234115508381 1.3500018124292416e-05
17.882894144707237 3.724627318673907 1.3497216500249358e-05
17.883894194709736 3.7250205168498915 1.3494415255698349e-05
17.884894244712235 3.7254137100304083 1.349161439063798e-05
17.885894294714735 3.725806898209532 1.348881390506683e-05
17.886894344717238 3.726200081381337

18.32791639581979 3.899021219179371 1.2288110709668826e-05
18.32891644582229 3.899411614991634 1.2285478007570527e-05
18.329916495824794 3.8998020032129497 1.2282845683047326e-05
18.330916545827293 3.900192383837586 1.2280213736092084e-05
18.331916595829792 3.90058275685981 1.2277582166697653e-05
18.33291664583229 3.90097312227389 1.2274950974856868e-05
18.33391669583479 3.9013634800740955 1.2272320160562557e-05
18.334916745837294 3.9017538302546955 1.226968972380753e-05
18.335916795839793 3.90214417280996 1.2267059664584587e-05
18.336916845842293 3.902534507734159 1.226442998288652e-05
18.337916895844792 3.902924835021563 1.2261800678706101e-05
18.33891694584729 3.9033151546664433 1.2259171752036095e-05
18.339916995849794 3.9037054666630717 1.2256543202869252e-05
18.340917045852294 3.9040957710057205 1.2253915031198311e-05
18.341917095854793 3.904486067688662 1.2251287237015997e-05
18.342917145857292 3.9048763567061697 1.2248659820315026e-05
18.343917195859795 3.9052666380525167 1.224

19.11395569778489 4.203076273058696 1.0334524768740797e-05
19.11495574778739 4.203458975116348 1.0332185672857608e-05
19.11595579778989 4.203841665238659 1.0329846945289267e-05
19.11695584779239 4.204224343420327 1.0327508586019573e-05
19.11795589779489 4.204607009656046 1.0325170595032314e-05
19.118955947797392 4.204989663940515 1.032283297231127e-05
19.11995599779989 4.2053723062684325 1.0320495717840208e-05
19.12095604780239 4.205754936634497 1.0318158831602884e-05
19.12195609780489 4.206137555033408 1.0315822313583048e-05
19.12295614780739 4.206520161459864 1.0313486163764432e-05
19.123956197809893 4.206902755908568 1.0311150382130766e-05
19.124956247812392 4.207285338374219 1.0308814968665762e-05
19.12595629781489 4.20766790885152 1.0306479923353126e-05
19.12695634781739 4.208050467335172 1.030414524617655e-05
19.12795639781989 4.20843301381988 1.0301810937119718e-05
19.128956447822393 4.208815548300346 1.0299476996166302e-05
19.129956497824892 4.209198070771275 1.0297143423299966

19.316965848292416 4.280512837084507 9.867216888123056e-06
19.317965898294915 4.280893008374699 9.864952213609014e-06
19.318965948297414 4.281273166665616 9.862687903907262e-06
19.319965998299917 4.281653311952082 9.860423958999498e-06
19.320966048302417 4.282033444228922 9.85816037886741e-06
19.321966098304916 4.28241356349096 9.855897163492682e-06
19.322966148307415 4.28279366973302 9.85363431285698e-06
19.323966198309915 4.283173762949929 9.851371826941964e-06
19.324966248312418 4.283553843136513 9.849109705729281e-06
19.325966298314917 4.283933910287599 9.846847949200573e-06
19.326966348317416 4.284313964398016 9.844586557337466e-06
19.327966398319916 4.28469400546259 9.842325530121581e-06
19.32896644832242 4.285074033476152 9.840064867534525e-06
19.329966498324918 4.2854540484335315 9.837804569557898e-06
19.330966548327417 4.285834050329559 9.835544636173288e-06
19.331966598329917 4.286214039159064 9.833285067362274e-06
19.332966648332416 4.286594014916879 9.831025863106422e-06
19

19.67098354917746 4.414243733303137 9.088175410494457e-06
19.67198359917996 4.414618984187788 9.086038663807386e-06
19.67298364918246 4.414994220281634 9.083902274842977e-06
19.67398369918496 4.415369441579735 9.08176624357952e-06
19.67498374918746 4.4157446480771485 9.079630569995298e-06
19.67598379918996 4.416119839768936 9.077495254068585e-06
19.67698384919246 4.416495016650158 9.075360295777644e-06
19.67798389919496 4.416870178715875 9.07322569510073e-06
19.67898394919746 4.41724532596115 9.07109145201609e-06
19.679983999199962 4.417620458381045 9.06895756650196e-06
19.68098404920246 4.417995575970623 9.066824038536567e-06
19.68198409920496 4.418370678724949 9.064690868098129e-06
19.68298414920746 4.418745766639087 9.062558055164856e-06
19.683984199209963 4.419120839708103 9.060425599714947e-06
19.684984249212462 4.419495897927064 9.058293501726592e-06
19.68598429921496 4.419870941291035 9.056161761177975e-06
19.68698434921746 4.420245969795085 9.054030378047268e-06
19.687984399219